covered topics 
- details about generator 
- generator *comprehension* / *func* / *class*
- bundled generators 
- coroutines

things I've missed 
- pros and cons about *generator*

### generators

count func

In [31]:
def count(start=0, step=1, end=10):
    n = start 
    while n <= end:
        yield n 
        n += step 

        
for x in count(10, 2.5, 15):
    x 

10

12.5

15.0

count class

In [32]:
class Count(object):
    def __init__(self, start=0, step=1, end=10):
        self.n    = start 
        self.step = step 
        self.end  = end  
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        n = self.n 
        if n > self.end:
            raise StopIteration()
            
        self.n += self.step 
        return n 
    
    
for x in Count(10, 2.5, 15):
    x 

10

12.5

15.0

yield, return 

In [33]:
def generator():
    yield "hey, Mr. Y"
    return 'returning from ... what'


g = generator()

try:
    next(g)  # result returned by `yield`
    next(g)  # the final was returned by `return` 
except StopIteration as err:
    err 

'hey, Mr. Y'

StopIteration('returning from ... what')

comprehension

In [22]:
list(
    x**3 for x in range(5)
)

[0, 1, 8, 27, 64]

order

In [53]:
def gene():
    
    print('Before 1')
    yield 1 
    
    print('After  1')
    print('Before 2')
    yield 2 
    
    print('After  2')
    print('Before 3')
    yield 3 
    
    print('After  3')
    
    '''
        if `return` or None:
            nothing returned 
        
        if `yield` 
            yield what u've given
    '''
    return 'hela '


g = gene()

for i in g:  # u can use `next` instead 
    i 

Before 1


1

After  1
Before 2


2

After  2
Before 3


3

After  3


pipelines

In [55]:
!cat ./src/p5_gen_pipe.txt | grep spam 

!cat ./src/p5_gen_pipe.txt | grep spam | sed 's/spam/bacon/g'

spam
spam spam 
spam spam spam 
bacon
bacon bacon 
bacon bacon bacon 


In [60]:
# recreating the pipe with generators :)

def cat(filename):
    for line in open(filename):
        yield line.rstrip()
        
def grep(sequence, search):
    for line in sequence:
        if search in line:
            yield line 
            
def replace(sequence, _search, _replace):
    for line in sequence:
        yield line.replace(_search, _replace)

In [69]:
seperated_steps = '''
    lines       = cat('./src/p5_gen_pipe.txt')
    spam_lines  = grep(lines, 'spam')
    bacon_lines = replace(spam_lines, 'spam', 'bacon')
'''

# compressed in one line XD

for line in replace(
    grep(cat('./src/p5_gen_pipe.txt'), 'spam'), 
    'spam',
    'bacon'
):
    print(line)

bacon
bacon bacon
bacon bacon bacon


tee - what does it do

In [97]:
import itertools as itr 

def spam_and_eggs():
    yield 'spam'
    yield 'eggs'
    

a, b, c = itr.tee(spam_and_eggs(), 3)

# the func is not that important 
#   the crucial one is the `n` (latter args)
#   it's like repeated genrators? (I think)
a, b, c 

[ _ for _ in a ]
[ _ for _ in b ]
[ _ for _ in c ]

(<itertools._tee at 0x10f250b48>,
 <itertools._tee at 0x10f262408>)

['spam', 'eggs']

['spam', 'eggs']

['spam', 'eggs']

tee - actual (sort of) implementataion

In [98]:
from collections import deque

def _tee(iterable, n=2):
    
    itr = iter(iterable)
    deq = [deque() for i in range(n)] 
    
    def gen(mydeq):
        
        while True:
            
            if not mydeq:
                
                try:
                    newval = next(itr)
                except StopIteration:
                    return 
                
                for d in deq:
                    d.append(newval)
            
            yield mydeq.popleft()
            
    return tuple(gen(d) for d in deq)


a, b, c = _tee(spam_and_eggs(), 3)

[i for i in a]
[i for i in b]
[i for i in c]

['spam', 'eggs']

['spam', 'eggs']

['spam', 'eggs']

powerset

In [120]:
import itertools as itr 

def powerset(seq):
    for size in range(1, len(seq) + 1):
        for item in itr.combinations(seq, size): 
            yield item 
            

for res in powerset('abc'):
    res 

('a',)

('b',)

('c',)

('a', 'b')

('a', 'c')

('b', 'c')

('a', 'b', 'c')

powerset - a simpler one

In [122]:
def powerset(seq):
    for size in range(1, len(seq) + 1):
        yield from itr.combinations(seq, size)  # for-yield => 1 line


for res in powerset('abc'):
    res

('a',)

('b',)

('c',)

('a', 'b')

('a', 'c')

('b', 'c')

('a', 'b', 'c')

flatten a list *recursively*

In [129]:
def flatten(seq):
    for item in seq:
        try:
            yield from flatten(item)
        except TypeError:
            yield item
            
            
list(flatten(
    [[], 1, [3, 5, 7], [9]]
))

[1, 3, 5, 7, 9]

context managers

In [170]:
# stage - init 

from datetime import datetime

from contextlib import (
    contextmanager, redirect_stdout,
    ExitStack
)


@contextmanager
def timer(name):
    
    start = datetime.now()
    yield 
    end   = datetime.now()
    
    print(f'{name} took {start-end}')

In [167]:
# stage - exp - ver1

@contextmanager
def write_to_log(name):
    with open(f'./src/p5_contx_{name}.txt', 'w') as fh:
        with redirect_stdout(fh):
            with timer(name):
                yield 
                
                
# then use it 

@write_to_log('some_func')
def some_func():
    print('hmm... hello?')
    print('and... hi?')
    
some_func()
 

!cat src/p5_contx_some_func.txt

hmm... hello?
and... hi?
some_func took -1 day, 23:59:59.999986


context managers - ```ExitStack```

In [169]:
# stage - exp - ver2

@contextmanager
def write_to_log(name):
    
    with ExitStack() as stack:
        
        fh = stack.enter_context(
            open('./src/p5_contx_exstack.txt', 'w'))
        
        stack.enter_context(redirect_stdout(fh))
        stack.enter_context(timer(name))
        
        yield 
        

@write_to_log('another func')
def what_the_func():
    print('hey yo')
    print('whats up')

what_the_func()


!cat src/p5_contx_exstack.txt

hey yo
whats up
another func took -1 day, 23:59:59.999979


In [190]:
# stage - exp - ver3 (maybe not)

# shortten the filename in code 
spam_file = './src/p5_contx_spam.txt'
eggs_file = './src/p5_contx_eggs.txt'


with ExitStack() as stack:
    spam_fh = stack.enter_context(open(spam_file, 'w'))
    eggs_fh = stack.enter_context(open(eggs_file, 'w'))
    
    spam_bytes_written = spam_fh.write('whoo~ spam')
    eggs_bytes_written = eggs_fh.write('whoo~ eggs')
    
    close_handlers = stack.pop_all().close
   

spam_bytes_written = spam_fh.write('yet another spam')
eggs_bytes_written = eggs_fh.write('and another eggs')


# then ...
close_handlers()

try:
    spam_bytes_written = spam_fh.write('cant write now')
except ValueError as err:
    err 

ValueError('I/O operation on closed file.')

### coroutines 

In [201]:
def gene():
    
    val = yield 'spam' 
    print(f'received val: {val}')  # "given" to val, not val
    
    yield f'previous val: {val}'
    

g = gene()

help(g.send)

Help on built-in function send:

send(...) method of builtins.generator instance
    send(arg) -> send 'arg' into generator,
    return next yielded value or raise StopIteration.

